In [17]:
import pandas as pd
import numpy as np
import requests
import json
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [18]:
# Configuring the driver to sign in properly so that it can run as fast as possible

opt = webdriver.FirefoxOptions()
opt.add_argument("--headless")
opt.add_argument("--disable-gpu")
opt.add_argument("--disable-dev-shm-usage")
opt.add_argument("--disable-extensions")
opt.add_argument("--disable-browser-side-navigation")
opt.add_argument("--enable-javascript")
opt.add_argument("--disable-infobars")
opt.add_argument("--window-size=1366,768")

In [19]:
# This actually instantiates the driver and opens the sign-in page for Geocaching
driver = webdriver.Firefox(options=opt)

driver.get('https://www.geocaching.com/account/signin?returnUrl=%2fplay')


In [20]:
# wait for page to load

wait = WebDriverWait(driver, 10)

In [ ]:
# Reading in my secrets

with open("secrets.txt") as f:
    lines = f.readlines()
    username = lines[0].strip()
    password = lines[1].strip()
    print(f"USERNAME={username}, PASSWORD={password}")

In [ ]:
# find and fill in login form
username = wait.until(EC.presence_of_element_located((By.ID, "UsernameOrEmail")))
password = wait.until(EC.presence_of_element_located((By.ID, "Password")))
username.send_keys(username)
password.send_keys(password)

In [ ]:
# submit login form

driver.find_element_by_id("SignIn").click()

# wait for page to fully load
time.sleep(5)

# navigate to the desired page
driver.get("https://www.geocaching.com/play")

In [ ]:

# close the browser
driver.quit()